<a href="https://colab.research.google.com/github/argonism/TsukurinagaraRL/blob/master/Zerokara_chap6_A2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

あれ、$\pi$と$Q$ってどう違うんだっけ？
行動確率と行動価値だから、根本的に違うかな...?

# A2C
- Advantage学習
  - 今まではQ学習に1step先までの状態をみてきたが、これを2step先以上までみる
  - $Q(s_t, a_t) \rightarrow R(t+1) + \gamma \bullet R(t+2) + (\gamma^2) \bullet max_a[Q(s_{t+2}, a)]$
- 分散学習(Asynchronous
  - 非同期に複数のエージェントを学習させる。
  - A2Cでは、一つのニューラルネットワークに対して、複数のエージェントにより学習を行い、分散学習する。
  - 学習が効率的になる
  - 自然とランダムなtransitionになるから、Experience Replayの必要がない
- Actor-Critic
  - これから詳しく。たしか、価値反復法と方策反復法を合わせたようなものみたいな説明だった気がする。

## Advantage学習
未来のstep数を増やせば増やすほど良いというわけではない
- 未来のstep数が増える = 未確定の行動を決定する回数が増える = 間違った学習をすることも増える

## 分散学習

## Actor-Critic
Actor-Criticのニューラルネットワークは、状態を受け取り、ActorとCriticを返す。

Actorは行動の数だけ出力され、Criticは状態価値をあらわす一つの値のみを返す。

ほうほう...

つまり、Actor-CriticのNNからは{行動の種類 + 1}個の出力がある。（Cartpoleなら3)
(+ 1は状態価値。critic分)

Actorの出力は、方策反復法と同じで状態$s_t$の入力に対してその行動がどれだけ良いものかを出力する。(ということは、一つの値が出力される？）
この出力をsoftmax関数を通せば、その行動の採用確率として利用できる。

critcは状態価値。その状態になった時に、その先で得られるであろう状態価値の割引報酬和。

Actor側で最大化したいのは状態$s_t$において、結合パラメータ$\theta$を使用して行動し続けた時の割引報酬和。この割引報酬和は方策勾配法を使って
$$J(\theta, s_t) = E[log\pi_\theta(a|s)(Q^\pi(s,a)-V_s^\pi)]$$
と表せる。（らしい）

- $\theta$ : それぞれの状態での行動方針（行動パターンの種類とも言える）
- $\pi$ : それぞれの状態での行動確率
- $E[]$: 期待値を計算する(実装時はミニバッチの平均として表される）
- $log\pi_\theta(a|s)$: 状態sのときに行動aを採用する確率のlog(行動確率のログ)
- $Q^\pi(s,a)$: 状態sで行動aを採用した場合の行動価値（定数として扱う）。Advantage学習で計算する
- $V_s^\pi$: criticの出力。状態価値

### 方策のエントロピー項
A3C, A2CではActorの学習ではさらに「方策のエントロピー項」を追加し、これは以下のように表される。
$$Actor_{entropy} = \sum^a[\pi_\theta(a|s)log\pi_\theta(a|s)]$$
Σは行動の種類についての総和を計算している。
方策が学習初期段階で、行動がランダムに決まるとき、最大となり、
方策で行動が一方に決まっているとき、最小になる。
Q. 目的を知った上で数式を見ても、ピンとこない。ランダムなときに最大になるのか...?

学習の初期段階で勾配の局所解に落ちるのを避ける狙い。

### Criticの学習
$$loss_{Critic} = (Q^\pi(s, a) - V_s^\pi)^2$$

この関数の最小化を目指して学習する。

状態価値をその状態での行動価値の総和かな？に近づけたいっぽくて、
確かにその状態で取れる行動の価値の総和がその状態の価値になるっているのは納得できるけど、
行動価値を正しく設定できていることが前提となるので、可能か？という気持ち




In [ ]:
# 使用するパッケージのインストール
# gym==0.17.2 pyvirtualdisplay==1.3.2
# xvfb=2:1.19.6-1ubuntu4.4 python-opengl=3.1.0+dfsg-1 ffmpeg=7:3.4.8-0ubuntu0.2
# JSAnimation==0.1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install JSAnimation > /dev/null 2>&1

In [ ]:
 import numpy as np
 import matplotlib.pyplot as plt
 %matplotlib inline
 import gym
from gym.wrappers import Monitor

In [ ]:
# 動画の描画関数の宣言
import glob
import io
import os
import base64
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython import display as ipythondisplay
from IPython.display import HTML
from pyvirtualdisplay import Display

display = Display(visible=0, size=(640, 400))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[-1]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def reset_video():
  mp4list = glob.glob('video/*.mp4')
  for mp4 in mp4list:
    os.remove(mp4)

def wrap_env(env):
  env = Monitor(env, './video', force=True, video_callable=(lambda ep: ep % 10 == 0))
  reset_video()
  return env

In [ ]:
ENV = 'CartPole-v0'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 1000

NUM_PROCESSES = 16
NUM_ADVANCED_STEP = 5
value_loss_coef = 0.5
entropy_coef = 0.01
max_grad_norm = 0.5

# Advantage学習のためのクラス

In [ ]:
class RolloutStorage(object):
  def __init__(self, num_steps, num_processes, obs_shape):
    self.observations = torch.zeros(num_steps + 1, num_processes, 4)
    # 各試行の終わりを示すための変数。次のステップが存在すれば1、なければ0
    self.masks = torch.ones(num_steps + 1, num_processes, 1)
    self.rewards = torch.zeros(num_steps, num_processes, 1)
    self.actions = torch.zeros(num_steps, num_processes, 1).long()

    self.returns = torch.zeros(num_steps + 1, num_processes, 1)
    self.index = 0
  
  def insert(self, current_obs, action, reward, mask):
    self.observations[self.index + 1].copy_(current_obs)
    self.masks[self.index + 1].copy_(mask)
    self.rewards[self.index].copy_(reward)
    self.actions[self.index].copy_(action)

    self.index = (self.index + 1) % NUM_ADVANCED_STEP
  
  def after_update(self):
    ''' advantage学習分のstep数を超えたら、改めて最初から入れ直す '''
    self.observations[0].copy_(self.observations[-1])
    self.masks[0].copy_(self.masks[-1])
  
  def compute_returns(self, next_value):
    ''' 割引報酬和を計算 '''
    self.returns[-1] = next_value
    # 5step目から計算していく
    for ad_step in reversed(range(self.rewards.size(0))):
      self.returns[ad_step] = self.returns[ad_step + 1] * \
        GAMMA * self.masks[ad_step + 1] + self.rewards[ad_step]


# ネットワーク
## evaluate_actions

> Actorの出力は、方策反復法と同じで状態 𝑠𝑡 の入力に対してその行動がどれだけ良いものかを出力する。(ということは、一つの値が出力される？） この出力をsoftmax関数を通せば、その行動の採用確率として利用できる。

の通り、actor_outputをsoftmaxに通したものが行動の確率変数となる。

その上で、log_softmax(x) -> log(softmax(x)) だから、

log_probs: $\log\pi_\theta(a|s)$

action_log_probs: 


In [ ]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self, n_in, n_mid, n_out):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_in, n_mid)
    self.fc2 = nn.Linear(n_mid, n_mid)
    self.actor = nn.Linear(n_mid, n_out)
    self.critic = nn.Linear(n_mid, 1)
  
  def forward(self, x):
    h1 = F.relu(self.fc1(x))
    h2 = F.relu(self.fc2(h1))
    critic_output = self.critic(h2)
    actor_output = self.actor(h2)

    return critic_output, actor_output
  
  def act(self, x):
    # なんじゃこれ...pytorch...
    value, actor_output = self(x)
    action_probs = F.softmax(actor_output, dim=1)
    # 多分確率変数(action_probs)の一番大きいやつを1つ取り出してくれる(0/1)
    action = action_probs.multinomial(num_samples=1)
    return action
  
  def get_value(self, x):
    value, actor_output = self(x)
    return value
  
  def evaluate_actions(self, x, actions):
    ''' ネットワーク更新時に使用 '''
    ''' epsilon-greedy法は使わず、確率的に行動を決める。 '''
    value, actor_output = self(x)
    log_probs = F.log_softmax(actor_output, dim=1)
    action_log_probs = log_probs.gather(1, actions)

    probs = F.softmax(actor_output, dim=1)
    entropy = - (log_probs * probs).sum(-1).mean()

    return value, action_log_probs, entropy


advantages: $Q^\pi(s,a)-V_s^\pi$

critic_loss: $(Q^\pi(s, a) - V_s^\pi)^2$

action_gain: $J(\theta, s_t) = E[log\pi_\theta(a|s)(Q^\pi(s,a)-V_s^\pi)]$


$$Actor_{entropy} = \sum^a[\pi_\theta(a|s)log\pi_\theta(a|s)]$$


In [ ]:
import torch
from torch import optim

class Brain(object):
  def __init__(self, actor_critic):
    self.actor_critic = actor_critic
    self.optimizer = optim.Adam(self.actor_critic.parameters(), lr=0.01)

  def update(self, rollouts):
    obs_shape = rollouts.observations.size()[2:]
    num_steps = NUM_ADVANCED_STEP
    num_processes = NUM_PROCESSES

    values, action_log_probs, entropy = self.actor_critic.evaluate_actions(
        rollouts.observations[:-1].view(-1, 4),
        rollouts.actions.view(-1, 1)
    )

    # rollouts.observationやactions.view(-1,1)は(process * advance_step)で 80x4, 80x1になる。

    values = values.view(num_steps, num_processes, 1)
    action_log_probs = action_log_probs.view(num_steps, num_processes, 1)

    # advantages: 行動価値 - 状態価値
    advantages = rollouts.returns[:-1] - values
    # criticのloss
    value_loss = advantages.pow(2).mean()
    # actorのgainを計算するらしいが、ところでgainって何？聞いたことないけど
    # J(theta, s_t)を計算してると思われる。action_log_probsがlog\pi(s|a)だし。meanで期待値か。
    action_gain = (action_log_probs * advantages.detach()).mean()
    # 誤差関数の総和。これなんだろ。criticとactionは独立して学習させるわけじゃないのか。
    # 説明に出てきてないと思われる
    total_loss = (value_loss * value_loss_coef - action_gain - entropy * entropy_coef)

    self.actor_critic.train()
    self.optimizer.zero_grad()
    total_loss.backward()
    nn.utils.clip_grad_norm_(self.actor_critic.parameters(), max_grad_norm)
    # 一気に結合パラメータが変化しすぎないように、勾配の大きさは最大0.5まで

    self.optimizer.step()

In [ ]:
import copy

class Environment:
  def run(self):
    ''' メインの実行 '''
    envs = [wrap_env(gym.make(ENV)) for i in range(NUM_PROCESSES)]

    n_in = envs[0].observation_space.shape[0]
    n_out = envs[0].action_space.n
    n_mid = 32
    actor_critic = Net(n_in, n_mid, n_out)

    global_brain = Brain(actor_critic)

    obs_shape = n_in
    current_obs = torch.zeros(NUM_PROCESSES, obs_shape)
    rollouts = RolloutStorage(NUM_ADVANCED_STEP, NUM_PROCESSES, obs_shape)
    episode_rewards = torch.zeros([NUM_PROCESSES, 1])
    final_rewards = torch.zeros([NUM_PROCESSES, 1])
    obs_np = np.zeros([NUM_PROCESSES, obs_shape])
    reward_np = np.zeros([NUM_PROCESSES, 1])
    done_np = np.zeros([NUM_PROCESSES, 1])
    each_step = np.zeros(NUM_PROCESSES)
    episode = 0

    obs = [envs[i].reset() for i in range(NUM_PROCESSES)]
    obs = np.array(obs)
    obs = torch.from_numpy(obs).float()
    current_obs = obs

    rollouts.observations[0].copy_(current_obs)

    for j in range(NUM_EPISODES * NUM_PROCESSES):
      # Advanced学習のためのループ
      for step in range(NUM_ADVANCED_STEP):
        with torch.no_grad():
          action = actor_critic.act(rollouts.observations[step])
        actions = action.squeeze(1).numpy()

        # 1step踏む
        for i in range(NUM_PROCESSES):
          obs_np[i], reward_np[i], done_np[i], _ = envs[i].step(actions[i])
          if done_np[i]:
            if i == 0:
              print(f'{episode} Episode: Finished after {each_step[i] + 1} time steps')
              episode += 1
            
            # 終了時の報酬の設定
            if each_step[i] < 195:
              reward_np[i] = -1.0
            else:
              reward_np[i] = 1.0
            
            each_step[i] = 0
            obs_np[i] = envs[i].reset()

          else:
            reward_np[i] = 0.0
            each_step[i] += 1
        # 1step終了

        reward = torch.from_numpy(reward_np).float()
        episode_rewards += reward

        # 最後のstepだったら0.0。それ以外なら1.0
        masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done_np])

        final_rewards *= masks

        final_rewards += (1 - masks) * episode_rewards

        episode_rewards *= masks

        current_obs *= masks

        obs = torch.from_numpy(obs_np).float()
        current_obs = obs

        rollouts.insert(current_obs, action.data, reward, masks)
    # Advanced学習のためのループ終了
      with torch.no_grad():
        next_value = actor_critic.get_value(rollouts.observations[-1]).detach()
      
      rollouts.compute_returns(next_value)

      global_brain.update(rollouts)
      rollouts.after_update()

      # 
      if final_rewards.sum().numpy() >= NUM_PROCESSES:
        print('連続成功')
        show_video()
        break
    



In [ ]:
cartpole_env = Environment()
cartpole_env.run()

0 Episode: Finished after 29.0 time steps
1 Episode: Finished after 16.0 time steps
2 Episode: Finished after 15.0 time steps
3 Episode: Finished after 19.0 time steps
4 Episode: Finished after 17.0 time steps
5 Episode: Finished after 22.0 time steps
6 Episode: Finished after 11.0 time steps
7 Episode: Finished after 61.0 time steps
8 Episode: Finished after 20.0 time steps
9 Episode: Finished after 34.0 time steps
10 Episode: Finished after 25.0 time steps
11 Episode: Finished after 18.0 time steps
12 Episode: Finished after 94.0 time steps
13 Episode: Finished after 47.0 time steps
14 Episode: Finished after 117.0 time steps
15 Episode: Finished after 86.0 time steps
16 Episode: Finished after 21.0 time steps
17 Episode: Finished after 21.0 time steps
18 Episode: Finished after 189.0 time steps
19 Episode: Finished after 139.0 time steps
20 Episode: Finished after 128.0 time steps
21 Episode: Finished after 131.0 time steps
22 Episode: Finished after 86.0 time steps
23 Episode: Fini